# Introduction #

In these exercises, you'll explore the operations a couple of popular convnet architectures use for feature extraction, learn about how convnets can capture large-scale visual features through stacking layers, and finally see how convolution can be used on one-dimensional data, in this case, a time series.

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex4 import *

import tensorflow as tf
import matplotlib.pyplot as plt
import visiontools

plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')

# Exploring Moving Windows #

This exercise is meant to give you an opportunity to explore the moving window computations. There aren't any right or wrong answers -- it's just a chance to experiment!

We've provided you with some images and kernels you can use. Run this cell to see them.

In [ ]:
from visiontools import edge, blur, bottom_sobel, emboss, sharpen, circle

image_dir = '/kaggle/input/computer-vision-resources/'
circle_64 = tf.expand_dims(circle([64, 64], val=1.0, r_shrink=4), axis=-1)
kaggle_k = visiontools.read_image(image_dir + str('k.jpg'), channels=1)
car = visiontools.read_image(image_dir + str('car_illus.jpg'), channels=1)
car = tf.image.resize(car, size=[200, 200])
images = [(circle_64, "circle_64"), (kaggle_k, "kaggle_k"), (car, "car")]

plt.figure(figsize=(14, 4))
for i, (img, title) in enumerate(images):
    plt.subplot(1, len(images), i+1)
    plt.imshow(tf.squeeze(img))
    plt.axis('off')
    plt.title(title)
plt.show();

kernels = [(edge, "edge"), (blur, "blur"), (bottom_sobel, "bottom_sobel"),
           (emboss, "emboss"), (sharpen, "sharpen")]
plt.figure(figsize=(14, 4))
for i, (krn, title) in enumerate(kernels):
    plt.subplot(1, len(kernels), i+1)
    visiontools.show_kernel(krn, digits=2, text_size=20)
    plt.title(title)
plt.show()

To choose one to experiment with, just enter it's name in the appropriate place below. Then, set the parameters for the window computation. Try out some different combinations and see what they do!

In [ ]:
# YOUR CODE HERE: choose an image
image = circle_64

# YOUR CODE HERE: choose a kernel
kernel = bottom_sobel

visiontools.show_extraction(
    image, kernel,

    # YOUR CODE HERE: set parameters
    conv_stride=1,
    conv_padding='valid',
    pool_size=2,
    pool_stride=2,
    pool_padding='same',
    
    subplot_shape=(1, 4),
    figsize=(14, 6),
)

Whenever you're ready to move on, just run this cell to mark the exercise as complete.

In [ ]:
q_1.check()

# The Receptive Field #

In all of the examples we've done, we've used $3 \times 3$ kernels; this is perhaps the most common choice for kernel size in convolutional networks. We might worry, though: if our images have dimension `(128, 128)`, are kernels with dimension `(3, 3)` large enough to capture all of the important features? A $3 \times 3$ square wouldn't cover the shape of an eye or an ear, for instance. Maybe we should use larger kernels?

In fact, this is rarely necessary. Networks occassionally will have an initial `Conv2D` layer with larger kernels, perhaps with `kernel_size=(5, 5)` or `(7, 7)`, but usually not much larger. Instead, convnets can more effectively capture large-scale information from an image by stacking convolution and pooling layers. This stacking increases the number of pixels the output neurons are receiving information from, that is, it increases the size of the neurons' **receptive field**. Let's see how this happens now.

### 2) How the Receptive Field Grows

This next picture illustrates two stacked convolutional layers both with `(3, 3)` kernels. The bottom layer represents the input. Each of the neurons in the first (middle) layer has a $3 \times 3$ receptive field. Following the path of connections, we can see that each of the neurons in the second (top) layer has a $5 \times 5$ receptive field.

<figure>
<img src="https://i.imgur.com/HmwQm2S.png" alt="Illustration of the receptive field of two stacked convolutions." width=250>
</figure>

If you added a *third* convolutional layer with a `(3, 3)` kernel, each of its neurons would have a receptive field of:

In [ ]:
# Lines below will give you a hint or solution
#_COMMENT_IF(PROD)_
q_2.a.hint()
#_COMMENT_IF(PROD)_
q_2.a.solution()

Now say you add a `(2, 2)` maximum pooling layer with `strides=2` after the third convolution. What receptive field do the outputs have now? (This is harder. Try the hint if you need help.)

In [ ]:
# Lines below will give you a hint or solution
#_COMMENT_IF(PROD)_
q_2.b.hint()
#_COMMENT_IF(PROD)_
q_2.b.solution()

# One-Dimensional Convolution #

Though we've been using convolutional networks on two-dimensional data, it turns out that they can also be useful on *one*-dimensional data, like time series or natural language texts. In fact, convolutional networks tend to be successful on any kind of data with a strong **local topological structure**, meaning that the information about a point tends to be concentrated in nearby points -- you can most successfully predict the value of a pixel by looking at nearby pixels, you can most successfully predict the weather today by looking at the weather yesterday instead of a month ago.

### 3) Apply a 1D Convolution

In this exercise, we'll see how a convolution can be used on a **time series**. The time series we'll use is from [Google Trends](https://trends.google.com/trends/); it measures the popularity of the search term "machine learning" for weeks from January 25, 2015 to January 15, 2020.

In [ ]:
import pandas as pd

# Load the time series as a Pandas dataframe
machinelearning = pd.read_csv(
    '/kaggle/input/computer-vision-resources/machinelearning.csv',
    parse_dates=['Week'],
    index_col='Week',
)

machinelearning.plot();

Because our data is one-dimensional, the kernel needs to be one-dimensional as well. Define a one dimensional kernel. Though not required, you'll get better results if the entries sum to 1.

In [ ]:
# YOUR CODE HERE: Define a 1D kernel. 
kernel = tf.constant([____])
q_3.check()

In [ ]:
#%%RM_IF(PROD)%%
kernel = tf.constant([0.1, 0.2, 0.3, 0.4])
q_3.assert_check_passed()

Now run the next cell to apply the kernel with a convolution and see what effect it had on the time series.

In [ ]:
# Reformat for TensorFlow
ts_data = machinelearning.to_numpy()
ts_data = tf.expand_dims(ts_data, axis=0)
ts_data = tf.cast(ts_data, dtype=tf.float32)
kern = tf.reshape(kernel, shape=(*kernel.shape, 1, 1))

ts_filter = tf.nn.conv1d(
    input=ts_data,
    filters=kern,
    stride=1,
    padding='VALID',
)

# Format as Pandas Series
machinelearning_filtered = pd.Series(tf.squeeze(ts_filter).numpy())

machinelearning_filtered.plot();

# Conclusion #

This lesson ends our discussion of feature extraction. Hopefully, having completed these lessons, you've gained some intuition about how the process works and why the usual choices for its implementation are often the best ones.

# Keep Going #

In the next lesson, [**Lesson 5**](#$NEXT_NOTEBOOK_URL$), you'll learn how to compose the `Conv2D` and `MaxPool2D` layers to build your own convolutional networks from scratch.